In [1]:
pwd

u'/home/dyros-vehicle/gitrepo/ims/ML_archive/GoogLeNet-Inception-tensorflow'

In [2]:
cd example/

/home/dyros-vehicle/gitrepo/ims/ML_archive/GoogLeNet-Inception-tensorflow/example


In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# File: pre_trained.py
# Author: Qian Ge <geqian1001@gmail.com>

import tensorflow as tf
import numpy as np
import scipy.misc
import argparse

from tensorcv.dataflow.image import ImageFromFile

import setup_env as conf
from nets.googlenet import GoogleNet
from utils.preprocess import resize_image_with_smallest_side, center_crop_image
from utils.classes import get_word_list


def display_data(dataflow, data_name):
    try:
        print('[{}] num of samples {}, num of classes {}'.
              format(data_name, dataflow.size(), len(dataflow.label_dict)))
    except AttributeError:
        print('[{}] num of samples {}'.
              format(data_name, dataflow.size()))
    print(dataflow._im_list)


def get_args():
    parser = argparse.ArgumentParser()

    parser.add_argument('--type', default='.jpg', type=str,
                        help='image file extension')
    return parser.parse_args()


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
model = GoogleNet(is_load=True, pre_train_path=conf.PARA_DIR)

image = tf.placeholder(tf.float32, shape=[None, None, None, 3])

test_data = ImageFromFile(
        # ed: FLAGS.type이 argparse라서 ipython에서 실행시키기 힘드므로 아래와 같이 한다
        # FLAGS.type,
        '.jpg',
        data_dir=conf.DATA_DIR,
        num_channel=3)

display_data(test_data, 'test_data')
word_dict = get_word_list('../data/imageNetLabel.txt')

model.create_model([image, 1])
test_op = tf.nn.top_k(tf.nn.softmax(model.layer['output']),
                          k=5, sorted=True)

input_op = model.layer['input']
writer = tf.summary.FileWriter(conf.SAVE_DIR)

[test_data] num of samples 3
['../data/IMG_4379.jpg' '../data/ILSVRC2017_test_00000004.jpg'
 '../data/IMG_7940.JPG']
**[warning]** consider use dictionary input.
Load conv1_7x7_s2 weights!
Load conv1_7x7_s2 biases!
Load conv2_3x3_reduce weights!
Load conv2_3x3_reduce biases!
Load conv2_3x3 weights!
Load conv2_3x3 biases!
Load inception_3a_1x1 weights!
Load inception_3a_1x1 biases!
Load inception_3a_3x3_reduce weights!
Load inception_3a_3x3_reduce biases!
Load inception_3a_3x3 weights!
Load inception_3a_3x3 biases!
Load inception_3a_5x5_reduce weights!
Load inception_3a_5x5_reduce biases!
Load inception_3a_5x5 weights!
Load inception_3a_5x5 biases!
Load inception_3a_pool_proj weights!
Load inception_3a_pool_proj biases!
Load inception_3b_1x1 weights!
Load inception_3b_1x1 biases!
Load inception_3b_3x3_reduce weights!
Load inception_3b_3x3_reduce biases!
Load inception_3b_3x3 weights!
Load inception_3b_3x3 biases!
Load inception_3b_5x5_reduce weights!
Load inception_3b_5x5_reduce biases!

In [5]:
test_op

TopKV2(values=<tf.Tensor 'TopKV2:0' shape=(?, 5) dtype=float32>, indices=<tf.Tensor 'TopKV2:1' shape=(?, 5) dtype=int32>)

In [8]:
model.layer

{'class_prob': <tf.Tensor 'class_prob:0' shape=(?, 1000) dtype=float32>,
 'conv_out': <tf.Tensor 'inception_5b_concat:0' shape=(?, ?, ?, 1024) dtype=float32>,
 'input': <tf.Tensor 'Placeholder:0' shape=(?, ?, ?, 3) dtype=float32>,
 'output': <tf.Tensor 'loss3_classifier/output:0' shape=(?, 1000) dtype=float32>,
 'pre_prob': <tf.Tensor 'pre_prob:0' shape=(?,) dtype=float32>,
 'prediction': <tf.Tensor 'ArgMax:0' shape=(?,) dtype=int64>}

In [5]:
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        writer.add_graph(sess.graph)

        for k in range(0, 50):
            if test_data.epochs_completed < 1:
                batch_data = test_data.next_batch()

                im = batch_data[0]
                im = resize_image_with_smallest_side(im, 224)

                # im = center_crop_image(im, 224, 224)
                # scipy.misc.imsave('{}test_{}.png'.format(conf.SAVE_DIR, k),
                #                   np.squeeze(im))
                result = sess.run(test_op, feed_dict={image: im})

                for val, ind in zip(result.values, result.indices):
                    print('val : ' , val)
                    print('ind : ' , ind)
                    print('class : ' , word_dict[ind[0]])
                    print('\n')


/usr/lib/python2.7/dist-packages/scipy/misc/pilutil.py:479: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/lib/python2.7/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


('val : ', array([0.40608373, 0.24211834, 0.04435861, 0.01696582, 0.01652327],
      dtype=float32))
('ind : ', array([285, 281, 282, 284, 673], dtype=int32))
('class : ', 'Egyptian cat')


('val : ', array([9.9577928e-01, 3.1153345e-03, 7.5795321e-04, 2.2768714e-04,
       3.9285191e-05], dtype=float32))
('ind : ', array([  1, 392, 397,   0, 390], dtype=int32))
('class : ', 'goldfish, Carassius auratus')


('val : ', array([9.8797357e-01, 9.3527948e-03, 1.5818434e-03, 7.9170836e-04,
       6.9607740e-05], dtype=float32))
('ind : ', array([829, 705, 874, 547, 565], dtype=int32))
('class : ', 'streetcar, tram, tramcar, trolley, trolley car')


